# Getting all NBA players links and names

**Imports**

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
import time
import re

**Main Script**

In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
url = 'https://www.nba.com/players'
driver.get(url)

# Find the toggler element
element = driver.find_elements(By.CSS_SELECTOR, '.Toggle_switch__kRCjc')
element = element[1]

# Get the position of the element
x = element.location['x']
y = element.location['y']

# Create an ActionChains object and move the cursor to the position of the element
action = ActionChains(driver)
action.move_by_offset(x, y)

# Click on the element
action.click()
action.perform()
time.sleep(1)

# Select 'All' in the dropdown list
select_element = driver.find_element(By.XPATH, '//*[@title="Page Number Selection Drown Down List"]')
select = Select(select_element)
select.select_by_visible_text('All')

time.sleep(1)

# Wait for the table to load
wait = WebDriverWait(driver, 15)
wait.until(lambda driver: len(driver.find_elements(By.XPATH, "//table//tbody/tr")) > 1)

# Read the data from the table
table = driver.find_element(By.XPATH, "//table")
data = pd.read_html(table.get_attribute('outerHTML'))[0]

# Find the player names and links
parent = table.find_elements(By.XPATH, "//td[contains(@class, 'primary') and contains(@class, 'text') and contains(@class, 'RosterRow_primaryCol__1lto4')]")
player_data = []
for index, row in data.iterrows():
    name = row['Player']
    formatted_name = re.sub(r"(\w)([A-Z])", r"\1 \2", name).title()
    link = parent[index].find_element(By.TAG_NAME, "a").get_attribute("href")
    
    if "career" not in link:
        player_id = link.split("/")[4]
        new_url = f'https://www.nba.com/stats/player/{player_id}/career'
        player_data.append([formatted_name, new_url])
    else:
        player_data.append([formatted_name, link])

# Save the data to a file
player_df = pd.DataFrame(player_data, columns=['Name', 'Link'])
player_df.to_csv('nba_players_2.csv', index=False)

# Close the driver
driver.quit()

**Showing the file:**

In [3]:
df = pd.read_csv('nba_players_2.csv')
df

,Name,Link
0,Alaa Abdelnaby,https://www.nba.com/stats/player/76001/career/
1,Zaid Abdul-Aziz,https://www.nba.com/stats/player/76002/career/
2,Kareem Abdul-Jabbar,https://www.nba.com/stats/player/76003/career/
3,Mahmoud Abdul-Rauf,https://www.nba.com/stats/player/51/career/
4,Tariq Abdul-Wahad,https://www.nba.com/stats/player/1505/career/
...,...,...
4815,Ante Zizic,https://www.nba.com/stats/player/1627790/career/
4816,Jim Zoet,https://www.nba.com/stats/player/78647/career/
4817,Bill Zopf,https://www.nba.com/stats/player/78648/career/
4818,Ivica Zubac,https://www.nba.com/stats/player/1627826/career/
